In [ ]:
!pip install wandb -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 74.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import kagglehub
import numpy as np
from scipy.stats import skew
import wandb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(
    project="07-project",
    name="experiment_1",
    config={
        "model": "RandomForestClassifier",
        "random_state": 8,
        "test_size": 0.2,
        "n_estimators": 100,
        "max_depth": None,
        "min_samples_split": 2
    }
)

wandb: Currently logged in as: nicolenb (nicolenb-really-great-tech). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# Download latest version
path = kagglehub.dataset_download("mujtabamatin/air-quality-and-pollution-assessment")
path += '/updated_pollution_dataset.csv'

print("Path to dataset files:", path)

100%|██████████| 84.1k/84.1k [00:00<00:00, 477kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mujtabamatin/air-quality-and-pollution-assessment/versions/2/updated_pollution_dataset.csv


In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,Moderate
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,Moderate
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,Moderate
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,Good
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,Good


In [ ]:
df.isnull().sum()

,0
Temperature,0
Humidity,0
PM2.5,0
PM10,0
NO2,0
SO2,0
CO,0
Proximity_to_Industrial_Areas,0
Population_Density,0
Air Quality,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
#removing rows that have negative values in columns

for i in ['SO2', 'PM10']:
 df = df[df[i] >= 0]
df.describe()

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density
count,4969.000000,4969.000000,4969.000000,4969.000000,4969.000000,4969.000000,4969.000000,4969.000000,4969.000000
mean,30.033628,70.047595,20.146750,30.233930,26.427289,10.082330,1.501133,8.417871,497.575770
std,6.727772,15.878042,24.567561,27.362723,8.907025,6.715513,0.546262,3.607820,152.931683
min,13.400000,36.000000,0.000000,0.000000,7.400000,0.000000,0.650000,2.500000,188.000000
25%,25.100000,58.300000,4.700000,12.400000,20.100000,5.100000,1.030000,5.400000,381.000000
50%,29.000000,69.800000,12.000000,21.700000,25.300000,8.100000,1.410000,7.900000,494.000000
75%,34.000000,80.300000,26.100000,38.100000,31.900000,13.800000,1.840000,11.100000,601.000000
max,58.600000,128.100000,295.000000,315.800000,64.900000,44.900000,3.720000,25.800000,957.000000


In [ ]:
numerical_columns = df.select_dtypes(exclude= 'object').columns

In [ ]:
#normalising the skewed features with sqrt transformation
for i in numerical_columns:
  df[f"Log_{i}"] = np.sqrt(df[i])
  print(f"Log_{i}: {skew(df[f'Log_{i}'])}")


Log_Temperature: 0.4557929835677416
Log_Humidity: 0.016121527811786104
Log_PM2.5: 1.0433402543506878
Log_PM10: 1.1035009551796635
Log_NO2: 0.21439098018274053
Log_SO2: 0.4092181458281077
Log_CO: 0.5709680478313724
Log_Proximity_to_Industrial_Areas: 0.07448928597959541
Log_Population_Density: -0.14265699110098493


In [ ]:
#encoding the multiclass target feature
target = df['Air Quality'].map({'Moderate': 0, 'Good': 1, 'Hazardous': 2, 'Poor': 3})

#Model Training

In [ ]:
X = df.drop(columns=['Air Quality'])
y = target

In [ ]:
config = wandb.config
test_size = config.get("test_size")
random_state = config.get("random_state")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

In [ ]:
model = RandomForestClassifier(random_state=config.get("random_state"), n_estimators=config.get("n_estimators"), max_depth=config.get("max_depth"), min_samples_split=config.get("min_samples_split"))
model.fit(X_train, y_train)

RandomForestClassifier(random_state=8)

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([1, 1, 3, 0, 0, 1, 1, 2, 0, 0, 0, 0, 1, 0, 3, 0, 1, 3, 2, 1, 3, 1,
       3, 2, 3, 0, 1, 3, 1, 0, 0, 2, 0, 0, 1, 3, 3, 3, 2, 0, 3, 0, 0, 0,
       1, 0, 0, 3, 0, 1, 1, 1, 0, 1, 3, 0, 1, 0, 1, 3, 3, 1, 2, 0, 3, 1,
       0, 3, 3, 1, 3, 1, 3, 0, 0, 2, 3, 1, 3, 1, 0, 0, 0, 0, 0, 1, 3, 2,
       1, 2, 3, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 3, 3, 1, 0, 2, 0, 1, 0, 2,
       2, 3, 1, 1, 3, 3, 1, 2, 1, 1, 1, 3, 0, 1, 2, 1, 3, 1, 3, 1, 3, 1,
       0, 1, 1, 3, 1, 0, 0, 0, 0, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       3, 0, 1, 3, 1, 1, 1, 0, 2, 2, 3, 1, 1, 2, 0, 0, 2, 1, 2, 0, 0, 0,
       0, 1, 1, 1, 3, 0, 1, 1, 3, 0, 0, 1, 1, 2, 2, 0, 1, 1, 1, 0, 1, 1,
       3, 0, 1, 3, 3, 1, 0, 0, 1, 3, 1, 3, 0, 3, 3, 3, 0, 3, 2, 1, 0, 2,
       1, 2, 3, 1, 1, 0, 0, 1, 3, 3, 1, 1, 1, 0, 2, 1, 0, 0, 1, 3, 1, 0,
       3, 1, 1, 0, 1, 0, 3, 3, 1, 0, 1, 0, 1, 3, 0, 1, 3, 1, 1, 0, 0, 0,
       2, 0, 0, 3, 0, 3, 1, 3, 3, 1, 1, 1, 1, 1, 2, 1, 1, 3, 0, 1, 1, 2,
       3, 0, 3, 3, 1, 3, 1, 1, 1, 0, 0, 3, 1, 0, 1,

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [ ]:
#logging in wandb
wandb.log({"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1})

In [ ]:
import pickle

model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as file:
  pickle.dump(model, file)

# Log the model to the W&B run
wandb.log_model(path=model_filename, name="RFC")

In [ ]:
wandb.finish()

accuracy,▁
f1,▁
precision,▁
recall,▁
accuracy,0.9497
f1,0.94928
precision,0.94926
recall,0.9497
